In [1]:
import pandas as pd
import numpy as np

In [3]:
df = pd.DataFrame([np.ones(1000000)]*11).T
df.head()

,0,1,2,3,4,5,6,7,8,9,10
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
3,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [4]:
%timeit [t for t in df.itertuples()]

1.5 s ± 10.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [5]:
%timeit [p for p in df.values]

700 ms ± 3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [6]:
from collections import namedtuple
Order = namedtuple("Order", ["funds", "order_id", "order_type", 
                             "price", "reason", "sequence",
                             "side", "size", "time", "type", "trader_id"], verbose=False, rename=False)   

%timeit [Order(*p) for p in df.values]

3.1 s ± 28.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
order = Order(*[1]*11)

In [8]:
%timeit order.side

59.8 ns ± 0.406 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [9]:
t = df.values[0, :]
%timeit t[0]

114 ns ± 8.14 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [10]:
%load_ext cython

In [22]:
%%cython --annotate

cimport numpy as np
cimport cython

@cython.boundscheck(False)
@cython.wraparound(False)
def loop_array(double [:, :] array): 
    cdef Py_ssize_t n = array.shape[0]
    for row in range(n):
        yield array[row, :]

In [14]:
%timeit [t for t in loop_array(df.values)]

926 ms ± 22.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [15]:
from numba import jit

@jit(nopython=True)
def loop_numba(array):
    for i in range(array.shape[0]):
        yield(array[i, :])

In [16]:
%timeit [t for t in loop_numba(df.values)]

845 ms ± 28.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
